# load package and model

In [9]:
%load_ext autoreload
%autoreload 2
import os
import sys
import logging
# sys.path

from ai_schema import *
from ai_schema.eval import eval_text_classify
from config_ai.backend import set_tf_config
from config_ai.utils import *
from config_ai.models.text_classify.tf_mlm_text_classify import *
from config_ai.models.text_classify.common import get_text_classify_output
from config_ai.data_utils import *
from config_ai.models import load_model
from config_ai.experiments import get_model_config

logging.basicConfig(level=logging.DEBUG, format="%(asctime)s [%(levelname)s][%(filename)s:%(lineno)d]:%(message)s",
                    datefmt='%Y-%m-%d %H:%M:%S')


os.environ["CUDA_VISIBLE_DEVICES"] = "1"
set_tf_config()



2021-11-04 14:34:50 [INFO][backend.py:57]:setting tensorflow config...
2021-11-04 14:34:50 [INFO][backend.py:61]:current devices:
2021-11-04 14:34:50 [INFO][backend.py:62]:cpus:[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
2021-11-04 14:34:50 [INFO][backend.py:63]:gpus:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2021-11-04 14:34:50 [INFO][backend.py:64]:setting gpu memory allow growth...
2021-11-04 14:34:50 [INFO][backend.py:67]:setting soft device placement...


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
config_path = "/nfs/pony/chenhao/workspace/mlm_cls/zero_shot1.ini"
config_path = "/nfs/pony/chenhao/workspace/mlm_cls/zero_shot2.ini"

# config_path = "/nfs/pony/chenhao/workspace/ontonotes_ner/tf_seq_labeling_config.ini"


config = read_config(config_path)
config
model_config = get_model_config(config)
model_config

model = TFMLMTextClassifyModel(config=model_config)
model.pattern
# model.word2label
# model.label2word
len(model.keep_tokens)
# model.tgt_tokens[:10]
model.tgt_tokens
model.tgt_token_ids





2021-11-02 17:49:36 [INFO][utils.py:287]:parsing config with path:/nfs/pony/chenhao/workspace/mlm_cls/zero_shot2.ini
2021-11-02 17:49:36 [INFO][utils.py:298]:loading base config...
2021-11-02 17:49:36 [INFO][utils.py:287]:parsing config with path:/nfs/pony/chenhao/workspace/ConfigAI/demo/tf_base_config.ini


{'common_config': {'experiment_dir': '/nfs/pony/chenhao/experiment',
  'project_name': 'sentiment',
  'is_train': False,
  'is_test': True,
  'is_save': False,
  'save_args': {'format': 'h5', 'tf_serving_version': 1},
  'eval_phase_list': ['dev', 'test'],
  'output_phase_list': ['dev', 'test'],
  'is_overwrite_experiment': True,
  'default_random_seed': 10,
  'base_config': '/nfs/pony/chenhao/workspace/ConfigAI/demo/tf_base_config.ini',
  'model_cls': 'TFMLMTextClassifyModel',
  'model_name': 'tf_mlm_text_classify_model'},
 'tokenizer_config': {'tokenizer_name': 'bert_word_piece',
  'tokenizer_args': {'vocabs': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt'}},
 'nn_model_config': {'pretrained_model_tag': 'bert',
  'pretrained_model_path': '/nfs/pony/chenhao/pretrain/chinese_roberta_wwm_ext_L-12_H-768_A-12',
  'h5_file': '/nfs/pony/chenhao/experiment/sentiment/tf_sentiment_mlm/model/nn_model/nn_model.h5'},
 'compile_config': {'optimizer_name': 'adam',
  'optimizer_ar

{'tokenizer_config': {'tokenizer_name': 'bert_word_piece',
  'tokenizer_args': {'vocabs': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt'}},
 'task_config': {'token2label_path': '/nfs/pony/chenhao/data/sentiment/text_classify/token2label.json',
  'max_len': 128,
  'pattern': '[MASK]理想。',
  'keep_token_path': '/nfs/pony/chenhao/data/sentiment/text_classify/keep_tokens_rbw.txt'},
 'model_name': 'tf_mlm_text_classify_model'}

2021-11-02 17:49:36 [INFO][core.py:41]:init model with config:
2021-11-02 17:49:36 [INFO][core.py:118]:initializing tokenizer with config:
{
    "tokenizer_name": "bert_word_piece",
    "tokenizer_args": {
        "vocabs": "/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt"
    }
}
2021-11-02 17:49:36 [INFO][core.py:126]:reinitializing tokenizer with keep_tokens
2021-11-02 17:49:36 [INFO][core.py:132]:tokenizer initialized with 6796 vocabs


'[MASK]理想。'

6796

['很', '不']

[1419, 127]

# process data

In [3]:
train_data_path = config['data_config']['train_data_path']
train_data = model.jload_lines(train_data_path)
f"{train_data_path}: {len(train_data)} items"



dev_data_path = config['data_config']['dev_data_path']
dev_data = model.jload_lines(dev_data_path)
f"{dev_data_path}: {len(dev_data)} items"


test_data_path = config['data_config']['test_data_path']
test_data = model.jload_lines(test_data_path)
f"{test_data_path}: {len(test_data)} items"



train_data[0]

'/nfs/pony/chenhao/data/sentiment/text_classify/train.jsonl: 16883 items'

'/nfs/pony/chenhao/data/sentiment/text_classify/dev.jsonl: 2111 items'

'/nfs/pony/chenhao/data/sentiment/text_classify/test.jsonl: 2111 items'

LabeledTextClassifyExample(text='贝贝好爱干净 每天出门都要洗澡 还喜欢喝蒙牛 不喜欢蹲地方 喜欢坐凳子上还喜欢和我坐在一起~', extra_text=None, label=Label(name='pos', prob=1.0))

In [4]:
data_manager = DataManager.get_instance(model=model, data=train_data)

In [5]:
features = data_manager.get_features()
feature = next(features)
feature

{'full_text': '[MASK]理想。贝贝好爱干净 每天出门都要洗澡 还喜欢喝蒙牛 不喜欢蹲地方 喜欢坐凳子上还喜欢和我坐在一起~',
 'text': '[MASK]理想。',
 'extra_text': '贝贝好爱干净 每天出门都要洗澡 还喜欢喝蒙牛 不喜欢蹲地方 喜欢坐凳子上还喜欢和我坐在一起~',
 'token_ids': [3,
  2,
  2643,
  1538,
  99,
  4,
  3806,
  3806,
  1041,
  2552,
  1332,
  455,
  2183,
  1008,
  477,
  4189,
  4071,
  3626,
  2284,
  2446,
  3974,
  828,
  2149,
  829,
  3459,
  2566,
  127,
  828,
  2149,
  3906,
  914,
  1880,
  828,
  2149,
  924,
  473,
  1121,
  125,
  3974,
  828,
  2149,
  735,
  1589,
  924,
  912,
  119,
  3859,
  71,
  4],
 'segment_ids': [0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'tokens': ['[CLS]',
  '[MASK]',
  '理',
  '想',
  '。',
  '[SEP]',
  '贝',
  '贝',
  '好',
  '爱',
  '干',
  '净',
  '每',
  '天',
  '出',
  '门',
  '都',
  '要',
  '洗',
  '澡',
  '还',
  '喜',
  '欢',
  '喝',
  '蒙',
  '牛',


In [6]:
records = data_manager.get_records(mode="train")
next(records)



{'idx': 0,
 'full_text': '[MASK]理想。贝贝好爱干净 每天出门都要洗澡 还喜欢喝蒙牛 不喜欢蹲地方 喜欢坐凳子上还喜欢和我坐在一起~',
 'text': '[MASK]理想。',
 'extra_text': '贝贝好爱干净 每天出门都要洗澡 还喜欢喝蒙牛 不喜欢蹲地方 喜欢坐凳子上还喜欢和我坐在一起~',
 'token_ids': [3,
  2,
  2643,
  1538,
  99,
  4,
  3806,
  3806,
  1041,
  2552,
  1332,
  455,
  2183,
  1008,
  477,
  4189,
  4071,
  3626,
  2284,
  2446,
  3974,
  828,
  2149,
  829,
  3459,
  2566,
  127,
  828,
  2149,
  3906,
  914,
  1880,
  828,
  2149,
  924,
  473,
  1121,
  125,
  3974,
  828,
  2149,
  735,
  1589,
  924,
  912,
  119,
  3859,
  71,
  4],
 'segment_ids': [0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'tokens': ['[CLS]',
  '[MASK]',
  '理',
  '想',
  '。',
  '[SEP]',
  '贝',
  '贝',
  '好',
  '爱',
  '干',
  '净',
  '每',
  '天',
  '出',
  '门',
  '都',
  '要',
  '洗',
  '澡',
  '还',
  '喜',
  '欢',
  '喝',
  '

# build&compile model

In [7]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

nn_model_args = dict(**config["nn_model_config"])
nn_model_args
model.build_model(**nn_model_args)



{'pretrained_model_tag': 'bert',
 'pretrained_model_path': '/nfs/pony/chenhao/pretrain/chinese_roberta_wwm_ext_L-12_H-768_A-12',
 'h5_file': '/nfs/pony/chenhao/experiment/sentiment/tf_sentiment_mlm/model/nn_model/nn_model.h5'}

2021-11-02 17:49:44.738344: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-02 17:49:44.745654: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3597740000 Hz
2021-11-02 17:49:44.746068: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55d8000c1ff0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-02 17:49:44.746084: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-11-02 17:49:44.866633: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55d80015c480 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-11-02 17:49:44.866667: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2021-11-02 17:49:44.867832: I tensorflow

In [9]:
compile_args = dict(**config["compile_config"])
compile_args
model.compile_model(**compile_args)


{'optimizer_name': 'adam', 'optimizer_args': {'learning_rate': 3e-05}}

2021-10-28 10:53:47 [INFO][tf_mlm_text_classify.py:64]:compiling model...
2021-10-28 10:53:47 [INFO][tf_core.py:124]:number of devices: 2, use <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f2baca44910>'s scope
2021-10-28 10:53:47 [INFO][losses.py:88]:build loss layer with loss function:<function sparse_categorical_crossentropy at 0x7f2bde2a70d0>
2021-10-28 10:53:47 [INFO][tf_mlm_text_classify.py:83]:training model's summary:
2021-10-28 10:53:47 [INFO][layer_utils.py:192]:Model: "train_model"
2021-10-28 10:53:47 [INFO][layer_utils.py:193]:__________________________________________________________________________________________________
2021-10-28 10:53:47 [INFO][layer_utils.py:190]:Layer (type)                    Output Shape         Param #     Connected to                     
2021-10-28 10:53:47 [INFO][layer_utils.py:195]:==================================================================================================
2021-10-28 10:53:47 [INFO][layer_

In [10]:
model.train_model.inputs
model.train_model.outputs
model.train_model.losses
model.train_model.metrics


AttributeError: 'NoneType' object has no attribute 'inputs'

In [11]:
model.get_dataset_info("train")
model.get_dataset_info("test")

({'token_ids': 'int32', 'segment_ids': 'int32', 'token_output': 'int32'},
 {'token_ids': (None,), 'segment_ids': (None,), 'token_output': (None,)})

({'token_ids': 'int32', 'segment_ids': 'int32'},
 {'token_ids': (None,), 'segment_ids': (None,)})

# train_model

In [12]:
tmp_data = train_data[:]
# tmp_data

train_args = dict(**config["train_config"])
train_args.update(batch_size=8, epochs=10, verbose=1)
train_args

model.train(train_data=train_data, **train_args)

{'epochs': 10, 'batch_size': 8, 'overwrite_cache': False, 'verbose': 1}

100%|██████████| 16883/16883 [00:06<00:00, 2688.88it/s]
2021-10-28 10:54:08 [INFO][tf_core.py:174]:train on 16883 tensors


Epoch 1/10


2021-10-28 10:54:15 [INFO][cross_device_ops.py:695]:batch_all_reduce: 201 all-reduces with algorithm = nccl, num_packs = 1
2021-10-28 10:54:16 [WARNING][cross_device_ops.py:724]:Efficient allreduce is not supported for 1 IndexedSlices
2021-10-28 10:54:16 [INFO][cross_device_ops.py:438]:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
2021-10-28 10:54:18 [INFO][cross_device_ops.py:438]:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
2021-10-28 10:54:18 [INFO][cross_device_ops.py:438]:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
2021-10-28 10:54:26 [INFO][cross_device_ops.py:695]:batch_all_reduce: 201 all-reduces with algorithm = nccl, num_packs = 1
2021-10-28 10:54:27 [WARNING][cross_device_ops.py:724]:Efficient al

  68/2111 [..............................] - ETA: 6:22 - accuracy: 0.8364 - loss: 0.4322

KeyboardInterrupt: 

# predict&evaluate

In [8]:
to_pred = train_data[:]
to_pred[:4]

pred = model.predict(to_pred, show_detail=False, batch_size=16)
pred[:4]


[LabeledTextClassifyExample(text='贝贝好爱干净 每天出门都要洗澡 还喜欢喝蒙牛 不喜欢蹲地方 喜欢坐凳子上还喜欢和我坐在一起~', extra_text=None, label=Label(name='pos', prob=1.0)),
 LabeledTextClassifyExample(text='感觉好像是文科生看一本《高等数学》的教材一样，流水账一般，只是背景很好罢了，选择在这样一个竞争激烈的时代，写了那么一个催人奋进的故事，文笔不咋地。', extra_text=None, label=Label(name='neg', prob=1.0)),
 LabeledTextClassifyExample(text='很安静,隔音设施不错.服务员态度很好,下次还会选这里', extra_text=None, label=Label(name='pos', prob=1.0)),
 LabeledTextClassifyExample(text='1 感觉外观还可以，符合我的要求，体积虽不算小，但比它大的翻盖手机还是很多的。2 比一张IC卡比较要小，厚度也还可以。键盘很漂亮，按键键盘面是平的，屏幕一般不会碰到按键，设计的很好。3 。。。一堆呢，不说了', extra_text=None, label=Label(name='pos', prob=1.0))]

100%|██████████| 16883/16883 [00:07<00:00, 2206.36it/s]
2021-11-02 17:50:07 [INFO][tf_core.py:205]:predicting with tf model...
2021-11-02 17:50:07.667771: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-02 17:50:43.422254: W tensorflow/core/common_runtime/bfc_allocator.cc:434] Allocator (GPU_0_bfc) ran out of memory trying to allocate 53.09MiB (rounded to 55672832)
Current allocation summary follows.
2021-11-02 17:50:43.422311: I tensorflow/core/common_runtime/bfc_allocator.cc:934] BFCAllocator dump for GPU_0_bfc
2021-11-02 17:50:43.422330: I tensorflow/core/common_runtime/bfc_allocator.cc:941] Bin (256): 	Total Chunks: 11, Chunks in use: 11. 2.8KiB allocated for chunks. 2.8KiB in use in bin. 41B client-requested in use in bin.
2021-11-02 17:50:43.422343: I tensorflow/core/common_runtime/bfc_allocator.cc:941] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-req

ResourceExhaustedError: OOM when allocating tensor with shape[16,128,6796] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Exp]

In [ ]:
from config_ai.models.text_classify.common import get_text_classify_output
from ai_schema.eval import eval_text_classify


output_data = get_text_classify_output(to_pred, pred)
output_data[:4]

In [17]:
true_labels = [e.label for e in to_pred]
eval_rs = eval_text_classify(true_labels, pred)
eval_rs

{'detail': {'neg': {'tp': 454,
   'fp': 143,
   'fn': 29,
   'precision': 0.7604690117252931,
   'recall': 0.9399585921325052,
   'f1': 0.8407407407407408},
  'pos': {'tp': 398,
   'fp': 29,
   'fn': 143,
   'precision': 0.9320843091334895,
   'recall': 0.7356746765249538,
   'f1': 0.8223140495867768}},
 'micro': {'tp': 852,
  'fp': 172,
  'fn': 172,
  'precision': 0.83203125,
  'recall': 0.83203125,
  'f1': 0.83203125},
 'macro': {'precision': 0.8462766604293913,
  'recall': 0.8378166343287294,
  'f1': 0.8420253979501104}}

# save&load

In [18]:
model_path = f"./models/{model.model_name}"
save_args = dict(**config["common_config"]["save_args"])
del save_args["tf_serving_version"]

save_args
model_path


model.save(path=model_path, **save_args)




{'format': 'h5'}

'./models/tf_mlm_text_classify_model'

2021-09-23 19:55:14 [INFO][core.py:73]:saving model to ./models/tf_mlm_text_classify_model
2021-09-23 19:55:14 [INFO][tf_core.py:56]:saving keras model to path:./models/tf_mlm_text_classify_model/nn_model/nn_model.h5
2021-09-23 19:55:16 [INFO][core.py:127]:save model done


In [20]:
loaded_model = load_model(path=model_path)
loaded_model

2021-09-23 19:56:18 [INFO][core.py:84]:loading model from path:./models/tf_mlm_text_classify_model
2021-09-23 19:56:18 [INFO][core.py:41]:init model with config:
2021-09-23 19:56:18 [INFO][core.py:44]:{
    "tokenizer_config": {
        "tokenizer_name": "bert_word_piece",
        "tokenizer_args": {
            "vocabs": [
                "[MASK]",
                "[PAD]",
                "[UNK]",
                "[CLS]",
                "[SEP]",
                "!",
                "\"",
                "#",
                "$",
                "%",
                "&",
                "'",
                "(",
                ")",
                "*",
                "+",
                ",",
                "-",
                ".",
                "/",
                "0",
                "1",
                "2",
                "3",
                "4",
                "5",
                "6",
                "7",
                "8",
                "9",
                ":",
 

2021-09-23 19:56:18 [INFO][core.py:118]:initializing tokenizer
2021-09-23 19:56:18 [INFO][tf_mlm_text_classify.py:60]:7390
2021-09-23 19:56:18 [INFO][core.py:118]:initializing tokenizer
2021-09-23 19:56:18 [INFO][tf_core.py:158]:loading keras model from path:./models/tf_mlm_text_classify_model/nn_model/nn_model.h5 with format:h5
2021-09-23 19:56:23 [INFO][layer_utils.py:192]:Model: "lml_model"
2021-09-23 19:56:23 [INFO][layer_utils.py:193]:__________________________________________________________________________________________________
2021-09-23 19:56:23 [INFO][layer_utils.py:190]:Layer (type)                    Output Shape         Param #     Connected to                     
2021-09-23 19:56:23 [INFO][layer_utils.py:195]:==================================================================================================
2021-09-23 19:56:23 [INFO][layer_utils.py:190]:token_ids (InputLayer)          [(None, None)]       0                                            
2021-09-23 19:56:23

In [21]:
loaded_pred = loaded_model.predict(data=to_pred)
loaded_pred

100%|██████████| 2/2 [00:00<00:00, 1279.73it/s]
2021-09-23 19:57:01 [INFO][tf_core.py:205]:predicting with tf model...
2021-09-23 19:57:02 [INFO][utils.py:103]:function:_model_predict cost:0.277 seconds
2021-09-23 19:57:02 [INFO][utils.py:103]:function:_post_predict cost:0.008 seconds


[Label(name='news_edu', prob=0.931022047996521),
 Label(name='news_edu', prob=0.9311829805374146)]